# Lightweight python components

Lightweight python components do not require you to build a new container image for every code change.
They're intended to use for fast iteration in notebook environment.

#### Building a lightweight python component
To build a component just define a stand-alone python function and then call kfp.components.func_to_container_op(func) to convert it to a component that can be used in a pipeline.

There are several requirements for the function:
* The function should be stand-alone. It should not use any code declared outside of the function definition. Any imports should be added inside the main function. Any helper functions should also be defined inside the main function.
* The function can only import packages that are available in the base image. If you need to import a package that's not available you can try to find a container image that already includes the required packages. (As a workaround you can use the module subprocess to run pip install for the required package.)
* If the function operates on numbers, the parameters need to have type hints. Supported types are ```[int, float, bool]```. Everything else is passed as string.
* To build a component with multiple output values, use the typing.NamedTuple type hint syntax: ```NamedTuple('MyFunctionOutputs', [('output_name_1', type), ('output_name_2', float)])```

In [1]:
EXPERIMENT_NAME = 'lightweight python components'
KFP_PACKAGE = 'https://storage.googleapis.com/ml-pipeline/release/0.1.16/kfp.tar.gz'

In [2]:
# Install the SDK
!pip3 install $KFP_PACKAGE --upgrade


    100% |████████████████████████████████| 153kB 24.8MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 2.4MB/s ta 0:00:011
    100% |████████████████████████████████| 1.4MB 12.0MB/s ta 0:00:01
    100% |████████████████████████████████| 2.1MB 8.2MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 20.9MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 21.3MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 21.9MB/s ta 0:00:01
  Stored in directory: /tmp/pip-ephem-wheel-cache-xmsmqhws/wheels/6e/b9/57/547b05d1f09ae798e899082c8ea2a7d693bde2e32f56f0792c
Successfully built kfp
fairing 0.5 has requirement google-auth>=1.6.2, but you'll have google-auth 1.6.1 which is incompatible.
fairing 0.5 has requirement google-cloud-storage>=1.13.2, but you'll have google-cloud-storage 1.13.0 which is incompatible.
fairing 0.5 has requirement kubernetes>=9.0.0, but you'll have kubernetes 8.0.0 which is incompatible.
  Found existing installation: g

In [3]:
import kfp.components as comp

Simple function that just add two numbers:

In [4]:
#Define a Python function
def add(a: float, b: float) -> float:
   '''Calculates sum of two arguments'''
   return a + b

Convert the function to a pipeline operation

In [5]:
add_op = comp.func_to_container_op(add)

A bit more advanced function which demonstrates how to use imports, helper functions and produce multiple outputs.

In [6]:
#Advanced function
#Demonstrates imports, helper functions and multiple outputs
from typing import NamedTuple
def my_divmod(dividend: float, divisor:float, output_dir:str = './') -> NamedTuple('MyDivmodOutput', [('quotient', float), ('remainder', float)]):
    '''Divides two numbers and calculate  the quotient and remainder'''
    #Imports inside a component function:
    import numpy as np

    #This function demonstrates how to use nested functions inside a component function:
    def divmod_helper(dividend, divisor):
        return np.divmod(dividend, divisor)

    (quotient, remainder) = divmod_helper(dividend, divisor)

    from tensorflow.python.lib.io import file_io
    import json
    
    # Exports a sample tensorboard:
    metadata = {
      'outputs' : [{
        'type': 'tensorboard',
        'source': 'gs://ml-pipeline-dataset/tensorboard-train',
      }]
    }
    with open(output_dir + 'mlpipeline-ui-metadata.json', 'w') as f:
      json.dump(metadata, f)

    # Exports two sample metrics:
    metrics = {
      'metrics': [{
          'name': 'quotient',
          'numberValue':  float(quotient),
        },{
          'name': 'remainder',
          'numberValue':  float(remainder),
        }]}

    with file_io.FileIO(output_dir + 'mlpipeline-metrics.json', 'w') as f:
        json.dump(metrics, f)

    from collections import namedtuple
    divmod_output = namedtuple('MyDivmodOutput', ['quotient', 'remainder'])
    return divmod_output(quotient, remainder)

Test running the python function directly

In [7]:
my_divmod(100, 7)

MyDivmodOutput(quotient=14, remainder=2)

#### Convert the function to a pipeline operation

You can specify an alternative base container image (the image needs to have Python 3.5+ installed).

In [8]:
divmod_op = comp.func_to_container_op(my_divmod, base_image='tensorflow/tensorflow:1.11.0-py3')

#### Define the pipeline
Pipeline function has to be decorated with the `@dsl.pipeline` decorator

In [9]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='Calculation pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
   a='a',
   b='7',
   c='17',
):
    #Passing pipeline parameter and a constant value as operation arguments
    add_task = add_op(a, 4) #Returns a dsl.ContainerOp class instance. 
    
    #Passing a task output reference as operation arguments
    #For an operation with a single return value, the output reference can be accessed using `task.output` or `task.outputs['output_name']` syntax
    divmod_task = divmod_op(add_task.output, b, '/')

    #For an operation with a multiple return values, the output references can be accessed using `task.outputs['output_name']` syntax
    result_task = add_op(divmod_task.outputs['quotient'], c)

#### Compile the pipeline

In [10]:
pipeline_func = calc_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

#### Submit the pipeline for execution

In [11]:
#Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}

#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

#vvvvvvvvv This link leads to the run information page. (Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working)

/opt/conda/lib/python3.6/site-packages/kfp/_client.py:194: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(f)


In [12]:
print("Previous completed run https://label-issues-0409.endpoints.code-search-demo.cloud.goog/pipeline/#/runs/details/7ce45f0d-7856-11e9-8964-42010a8e00ff")
print("Previous experiment https://label-issues-0409.endpoints.code-search-demo.cloud.goog/pipeline/#/experiments/details/82687784-bdd0-4c8d-9bc5-ea9683522735")

Previous completed run https://label-issues-0409.endpoints.code-search-demo.cloud.goog/pipeline/#/runs/details/7ce45f0d-7856-11e9-8964-42010a8e00ff
